# Estrutura de ficheiros utilizada para desenvolver o agente

### O agente está distribuído por 5 ficheiros:

+ `load.py` : Responsável pelo parser e a importação dos dados pelo csv.
+ `data_config.py` : Responsável por carregar todos os dados importados pelo `load.py` e, em seguida, distribui cada unidade curricular por uma turma e um professor de forma cíclica, criando também uma função que verifica, com base nesses dados, os dias e horas em que cada professor está disponível.
+ Conjunto de restrições:
    + `hard_constraints_def.py` : Responsável por definir as funções que verificam as *hard constraints* que o agente tem de respeitar.
    + `soft_constraints_def.py` : Responsável por definir as funções que verificam as *soft constraints* que o agente pode, ou não, respeitar, mas é preferível que ele respeite.
+ `main.py` : Responsável por chamar todos os outros ficheiros e montar o agente em si utilizando a biblioteca **Constraint** de python. 


In [ ]:
# Ficheiro: Load.py

import csv

def load_csv(path):
    data = []
    with open(path, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data

def load_courses(path='dados/cursos.csv'):
    cursos = load_csv(path)
    for c in cursos:
        c['id'] = int(c['id'])
        c['n_turmas'] = int(c['n_turmas'])
    return cursos

def load_teachers(path='dados/professores.csv'):
    professores = load_csv(path)
    for p in professores:
        p['id'] = int(p['id'])
    return professores

def load_rooms(path='dados/salas.csv'):
    salas = load_csv(path)
    for s in salas:
        s['id'] = int(s['id'])
    return salas

def load_curricular_units(path='dados/unidades_curriculares.csv'):
    ucs = load_csv(path)
    seen = set()
    ucs_unique = []
    for u in ucs:
        if u['id'] not in seen:
            u['id'] = int(u['id'])
            u['curso_id'] = int(u['curso_id'])
            u['n_aulas_semana'] = int(u['n_aulas_semana'])
            ucs_unique.append(u)
            seen.add(u['id'])
    return ucs_unique

def load_availabilities(path='dados/disponibilidades.csv'):
    disp = load_csv(path)
    for d in disp:
        d['prof_id'] = int(d['prof_id'])
        d['hora_inicio'] = int(d['hora_inicio'])
        d['hora_fim'] = int(d['hora_fim'])
    return disp

def load_classes(path='dados/turmas.csv'):
    turmas = load_csv(path)
    seen = set()
    turmas_unique = []
    for t in turmas:
        if t['id'] not in seen:
            t['id'] = int(t['id'])
            t['curso_id'] = int(t['curso_id'])
            turmas_unique.append(t)
            seen.add(t['id'])
    return turmas_unique

def load_all():
    return {
        'cursos': load_courses(),
        'professores': load_teachers(),
        'salas': load_rooms(),
        'unidades_curriculares': load_curricular_units(),
        'turmas': load_classes(),
        'disponibilidades': load_availabilities()
    }

if __name__ == "__main__":
    dados = load_all()
    print("=== Cursos ===")
    for c in dados['cursos']:
        print(c)

    print("\n=== Professores ===")
    for p in dados['professores']:
        print(p)

    print("\n=== Salas ===")
    for s in [s['nome'] for s in dados['salas']]:
        print(s)

    print("\n=== Turmas ===")
    for t in dados['turmas']:
        print(t)

    print("\n=== Unidades Curriculares ===")
    for u in dados['unidades_curriculares']:
        print(u)

    print("\n=== Disponibilidades ===")
    for d in dados['disponibilidades']:
        print(d)


In [ ]:
# Ficheiro: data_config.py

from itertools import product
from load import *

dados = load_all()

# Configuração básica
dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
horas = [9, 11, 14, 16]
salas = [s['nome'] for s in dados['salas']]
turmas = [t['id'] for t in dados['turmas']]
ucs = [uc['id'] for uc in dados['unidades_curriculares']]
professores = [p['id'] for p in dados['professores']]

uc_to_turma = {}
uc_to_professor = {}

# Distribuir UCs pelas turmas
for i, uc in enumerate(dados['unidades_curriculares']):
    uc_id = uc['id']
    # Alternar entre turmas (0 → turma1, 1 → turma2, etc.)
    turma_index = i % len(turmas)
    uc_to_turma[uc_id] = turmas[turma_index]
    
    # Distribuir professores
    prof_index = i % len(professores)
    uc_to_professor[uc_id] = professores[prof_index]

# Disponibilidades do CSV
disponibilidades = dados['disponibilidades']

def check_professor_availability(prof_id):
    disponiveis = []
    for d, h in product(dias, horas):
        for disp in disponibilidades:
            if (disp['prof_id'] == prof_id and 
                disp['dia'] == d and 
                disp['hora_inicio'] <= h < disp['hora_fim']):
                disponiveis.append((d, h))
                break
    return disponiveis

In [ ]:
# Ficheiro: hard_constraints_def.py

# Conflitos de sala
def no_same_room_same_time(a1, a2):
    return not (a1[0]==a2[0] and a1[1]==a2[1] and a1[2]==a2[2])

# Máximo 3 aulas/dia por turma
def max_three_per_day_turma(*aulas):
    count_por_dia_turma = {}
    for aula in aulas:
        dia, _, _, _, turma, _ = aula
        key = f"{turma}_{dia}"
        count_por_dia_turma[key] = count_por_dia_turma.get(key,0)+1
        if count_por_dia_turma[key]>3: return False
    return True

# Aulas da mesma UC em dias diferentes
def same_uc_different_days(a1,a2):
    return a1[0] != a2[0] if a1[5]==a2[5] else True

# Cada UC → 2 aulas
def exactly_two_per_uc(*aulas, ucs=None):
    count = {uc:0 for uc in ucs}
    for aula in aulas:
        count[aula[5]] += 1
    return all(v==2 for v in count.values())

# Cada turma → 10 aulas
def exactly_ten_per_turma(*aulas, turmas=None):
    count = {t:0 for t in turmas}
    for aula in aulas:
        count[aula[4]] += 1
    return all(v==10 for v in count.values())

# Cada aula → 2 horas
def check_duration(aula):
    _, hora, _, _, _, _ = aula
    blocos_validos = [9, 11, 14, 16]
    return hora in blocos_validos

In [ ]:
# Ficheiro: soft_constraints_def.py

# A mesma UC tem de ter aulas em dias diferentes
def check_distinct_day_classes(*aulas):
    uc_days = {}
    for aula in aulas:
        dia, _, _, _, _, uc = aula
        uc_days.setdefault(uc, set()).add(dia)
    return sum(len(dias) - 2 for dias in uc_days.values() if len(dias) > 2) == 0

# A mesma turma só pode ter até 4 dias de aulas
def check_weekly_days(*aulas):
    turma_days = {}
    for aula in aulas:
        dia, _, _, _, turma, _ = aula
        turma_days.setdefault(turma, set()).add(dia)
    return all(len(dias) <= 4 for dias in turma_days.values())

# As aulas da mesma UC não podem ser consecutivas
def check_consecutive_classes(*aulas):
    dia_to_horas_turma = {}
    for aula in aulas:
        dia, hora, _, _, turma, _ = aula
        dia_to_horas_turma.setdefault((turma, dia), []).append(hora)

    def hora_to_int(h):
        if isinstance(h, int):
            return h * 60
        elif isinstance(h, str):
            partes = h.split(':')
            if len(partes) == 2:
                hh, mm = map(int, partes)
                return hh * 60 + mm
            else:
                return int(h) * 60
        else:
            raise ValueError(f"Formato de hora inesperado: {h}")

    for (turma, dia), horas in dia_to_horas_turma.items():
        horas_ordenadas = sorted(hora_to_int(h) for h in horas)
        for i in range(1, len(horas_ordenadas)):
            if horas_ordenadas[i] - horas_ordenadas[i-1] != 60:
                return False
    return True

# Verifica se cada turma usa no máximo 3 salas diferentes nas aulas fornecidas
def check_different_classes(*aulas):
    turma_salas = {}
    for aula in aulas:
        _, _, sala, _, turma, _ = aula
        turma_salas.setdefault(turma, set()).add(sala)
    return all(len(salas) <= 3 for salas in turma_salas.values())



In [ ]:
# Ficheiro: main.py

from constraint import Problem
from data_config import *
from hard_constraints_def import *
from soft_constraints_def import *

# Criar problema CSP
problem = Problem()
all_vars = []

# Atribui 2 aulas por UC
for uc in ucs:
    turma = uc_to_turma[uc]
    prof = uc_to_professor[uc]
    disp = check_professor_availability(prof)
    dominio = [(d, h, s, prof, turma, uc) for s in salas for d, h in disp]
    problem.addVariable(f"UC{uc}_A1", dominio)
    problem.addVariable(f"UC{uc}_A2", dominio)
    all_vars.extend([f"UC{uc}_A1", f"UC{uc}_A2"])

# === Restrições rígidas ===
for i in range(len(all_vars)):
    for j in range(i + 1, len(all_vars)):
        problem.addConstraint(no_same_room_same_time, (all_vars[i], all_vars[j]))

for t in turmas:
    vars_t = [v for v in all_vars if uc_to_turma[int(v[2:v.find('_')])] == t]
    problem.addConstraint(max_three_per_day_turma, vars_t)

for uc in ucs:
    problem.addConstraint(same_uc_different_days, (f"UC{uc}_A1", f"UC{uc}_A2"))

for var in all_vars:
    problem.addConstraint(check_duration, [var])

problem.addConstraint(lambda *a: exactly_two_per_uc(*a, ucs=ucs), all_vars)
problem.addConstraint(lambda *a: exactly_ten_per_turma(*a, turmas=turmas), all_vars)

# Gerar até 200 soluções válidas
print("🧩 A gerar soluções válidas...")
solucoes = []
for sol in problem.getSolutionIter():
    solucoes.append(sol)
    if len(solucoes) >= 200:
        break

if not solucoes:
    print("❌ Nenhuma solução encontrada")
    exit()

print(f"✅ Encontradas {len(solucoes)} soluções válidas")

# Avaliar cada solução com soft constraints
def pontuacao(sol):
    aulas = list(sol.values())
    score = 0
    if check_distinct_day_classes(*aulas): score += 1
    if check_weekly_days(*aulas): score += 1
    if check_consecutive_classes(*aulas): score += 1
    if check_different_classes(*aulas): score += 1
    return score

avaliadas = [(sol, pontuacao(sol)) for sol in solucoes]
avaliadas.sort(key=lambda x: x[1], reverse=True)

melhor_sol, melhor_score = avaliadas[0]
print(f"🏆 Melhor solução encontrada com pontuação: {melhor_score}/4\n")

# Visualizar melhor solução
for t in turmas:
    print(f"📘 Turma {t}")
    tabela = {d: {h: "" for h in horas} for d in dias}
    for val in melhor_sol.values():
        d, h, sala, prof, turma, uc = val
        if int(turma) == int(t):
            tabela[d][h] = f"UC{uc} ({sala}, Prof {prof})"

    print(f"{'Hora':<6}" + ''.join(f"{d:<22}" for d in dias))
    for h in horas:
        print(f"{h:<6}" + ''.join(f"{tabela[d][h]:<22}" for d in dias))
    print()
